In [2]:
import pandas as pd
import numpy as np

import math
import os

# # Make TensorFlow logs less verbose
# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf

import flwr as fl
import tensorflow_probability as tfp

import keras
from sklearn.model_selection import train_test_split
from utils import model as mod
from utils import datapath
from flwr.common import NDArrays, Scalar
from scipy.stats import pearsonr
from typing import Dict, Optional, Tuple, List, Union
import typing
import random
import json
np.random.seed(2048)
tf.random.set_seed(2048)
random.seed(2048)
tf.keras.utils.set_random_seed(2048)



NUM_EPOCHS=10
BATCH_SIZE=16
path='resultsbn'
strat='FedBN_Aug'

In [3]:
!pwd

/rds/user/sc2427/hpc-work/Final/FL


In [4]:
data=pd.read_csv(datapath)

In [5]:
df = pd.DataFrame(columns = data.columns)


In [6]:

dfs_to_concat = []

for i in range(250, 1000):
    mean_data = data.loc[data['Stamp'] == i].mean()
    dfs_to_concat.append(mean_data)

result_df = pd.concat(dfs_to_concat, axis=1).T.reset_index(drop=True)

In [7]:
df=result_df

In [8]:
def gaussian_noise(x,mu,std):
    noise = np.random.normal(mu, std, size = x.shape)
    for i in range(3,30):
      x[i]=x[i]+noise[i]
    return x

In [9]:
data_new = pd.DataFrame(columns = data.columns)
for i in range(df.shape[0]):
  data_new=pd.concat([data_new,df.loc[i].to_frame().T])
  data_new=pd.concat([data_new,gaussian_noise(df.iloc[i], 0, 0.01).to_frame().T])
  data_new=pd.concat([data_new,gaussian_noise(df.iloc[i], 0, 0.01).to_frame().T])

  data_new=pd.concat([data_new,gaussian_noise(df.iloc[i], 0, 0.01).to_frame().T])
  data_new=pd.concat([data_new,gaussian_noise(df.iloc[i], 0, 0.01).to_frame().T])

  data_new=pd.concat([data_new,gaussian_noise(df.iloc[i], 0, 0.01).to_frame().T])
  data_new=pd.concat([data_new,gaussian_noise(df.iloc[i], 0, 0.01).to_frame().T])
  data_new=pd.concat([data_new,gaussian_noise(df.iloc[i], 0, 0.01).to_frame().T])

  data_new=pd.concat([data_new,gaussian_noise(df.iloc[i], 0, 0.01).to_frame().T])




In [10]:
data_new.shape

(6750, 38)

In [11]:
y_labels=df.columns[-8:]

In [12]:
x_labels=df.columns[1:-8]

In [13]:
x=np.array(data_new[x_labels])

In [14]:
y=np.array(data_new[y_labels])

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1)

In [16]:
x_train.shape

(5062, 29)

In [17]:
# y_train=y_train.reshape((1,y_train.shape[0],y_train.shape[1]))
y_train.shape

(5062, 8)

In [18]:

model_eval = mod()
model_eval.compile(keras.optimizers.Adam(0.01), loss=tf.keras.metrics.mean_squared_error, metrics=[tf.keras.metrics.RootMeanSquaredError() ])

2023-09-27 15:41:29.015116: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [19]:
def pcc(y_val, y_pred):
  temp={}
  for i in range(len(y_labels)):
    temp[y_labels[i]]=(pearsonr(y_val[:,i],y_pred[:,i])[0])
  return sum(list(temp.values()))/len(list(temp.values()))

In [20]:
class FlowerClient(fl.client.NumPyClient):

    def __init__(self, model,cid, x_train, y_train, x_val, y_val, path) -> None:
        self.model = model
        self.cid=cid
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val
        self.path=path

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        # loss, acc = self.model.evaluate(self.x_val, self.y_val, verbose=2)
        # loss, rmse = model.evaluate(self.x_val, self.y_val)
        loss,rmse=self.model.evaluate(self.x_val, self.y_val)

        # result["pr"].append(pr)


        try:
          data=pd.read_csv(f"{self.path}/result_{self.cid}.csv")
          data.drop(["Unnamed: 0"], axis=1, inplace=True)

        except:
          data=pd.DataFrame(columns=["round","Loss", "RMSE"])

        data=pd.concat([data,pd.Series([config['server_round'], loss, rmse ] , index=data.columns ).to_frame().T])
        data.to_csv(f"{self.path}/result_{self.cid}.csv")

        return loss, len(self.x_val), {"accuracy": rmse}

    def set_parameters(self, parameters: List[np.ndarray])-> None:
        self.model.train()
        keys = [k for k in self.model.state_dict().keys() if "bn" not in k]
        params_dict = zip(keys, parameters)
        state_dict = OrderedDict({k: tf.Tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=False)

In [21]:

#import pearsonr
def get_eval_fn(model, result,result_train):
    """Return an evaluation function for server-side evaluation."""

    # Load data and model here to avoid the overhead of doing it in `evaluate` itself


    # The `evaluate` function will be called after every round
    def evaluate(server_round: int, weights: fl.common.NDArrays, config: Dict[str, Scalar]) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        model.set_weights(weights)  # Update model with the latest parameters
        loss, rmse = model.evaluate(x_test, y_test)
        loss_train,rmse_train=model.evaluate(x_train, y_train)
        result_train['Losses'].append(loss_train)
        result_train['rmse'].append(rmse_train)

        result["Losses"].append(loss)
        result["rmse"].append(rmse)
        # result["pr"].append(pr)
        for i in range(len(y_labels)):
          result["pearson"][y_labels[i]].append(pearsonr(y_test[:,i],model.predict(x_test)[:,i])[0])
          result_train['pearson'][y_labels[i]].append(pearsonr(y_train[:,i],model.predict(x_train)[:,i])[0])


        return loss, {"accuracy": rmse}

    return evaluate

In [22]:
model_eval.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                480       
                                                                 
 batch_normalization (Batch  (None, 16)                64        
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 batch_normalization_1 (Bat  (None, 16)                64        
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 1016 (3.97 KB)
Trainable params: 952 (3.72

In [23]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
    }
    return config

def evaluate_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
    }
    return config

In [24]:
ray_init_args = {
            "num_gpus": 0,
            "num_cpus": 3
        }
client_res={"num_gpus": 0, "num_cpus": 3}

In [ ]:
inital=mod()
#for clients from 2 to 10, run the strategy for 10 rounds each and create a list of results
results=[]
results_train=[]
# for i in range(5):
for i in range(2,11):
    try:
        os.mkdir(f"{path}/{i}")
    except:
        print()
    try:
        os.mkdir(f"{path}/{i}/{strat}")
    except:
        print()
    
    NUM_CLIENTS = i
    # losses=[]
    def client_fn(cid: str) -> fl.client.Client:
        # Create model
        model = mod()
        model.compile(keras.optimizers.Adam(0.01), loss=tf.keras.metrics.mean_squared_error, metrics=[tf.keras.metrics.RootMeanSquaredError()])    
        partition_size = math.floor(len(x_train) / NUM_CLIENTS)
        idx_from, idx_to = int(cid) * partition_size, (int(cid) + 1) * partition_size
        full_x_train_cid = x_train[idx_from:idx_to]
        full_y_train_cid = y_train[idx_from:idx_to]
    
    
        partition_size = math.floor(len(x_test) / NUM_CLIENTS)
        idx_from, idx_to = int(cid) * partition_size, (int(cid) + 1) * partition_size
        full_x_test_cid = x_test[idx_from:idx_to]
        full_y_test_cid = y_test[idx_from:idx_to]
    
        # Create and return client
        return FlowerClient(model,cid, full_x_train_cid, full_y_train_cid, full_x_test_cid, full_y_test_cid, f"{path}/{i}/{strat}")


    
    
    result={ 'clients':i  , 'Losses':[], "rmse":[], 'pearson':{y_labels[i]:[] for i in range(len(y_labels))}}
    
    result_train={ 'clients':i  , 'Losses':[], "rmse":[], 'pearson':{y_labels[i]:[] for i in range(len(y_labels))}}
    
    try:
        del strategy
    except:
        print('starting')
    # Create FedAvg strategy
    strategy = fl.server.strategy.FedAvg(
      min_fit_clients=i,  # Never sample less than 10 clients for training
      evaluate_fn=get_eval_fn(model_eval, result,result_train),
      on_fit_config_fn=fit_config,
      on_evaluate_config_fn=evaluate_config,
      initial_parameters=fl.common.ndarrays_to_parameters(inital.get_weights()),
    )
    
    # Start simulation
    fl.simulation.start_simulation(
      client_fn=client_fn,
      num_clients=NUM_CLIENTS,
      config=fl.server.ServerConfig(num_rounds=10),
      strategy=strategy,
      ray_init_args=ray_init_args,
      client_resources=client_res,
    )
    results.append(result)
    results_train.append(result_train)
    # print(result)
    # try:
    #     data=pd.read_csv(f"{path}/combine_random{result['clients']}.csv")
    #     data.drop(["Unnamed: 0"], axis=1, inplace=True)
        
    # except:
    #     data=pd.DataFrame(columns=["Method", "Loss_Task1", "RMSE_Task1"])
    # data=pd.concat([data,pd.Series(["FedAvgEWC_Aug", result['Losses'][-1],result['rmse'][-1]] , index=data.columns ).to_frame().T])
    # data.to_csv(f"{path}/combine_random{result['clients']}.csv")



INFO flwr 2023-09-27 15:41:29,362 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)



starting


2023-09-27 15:41:32,723	ERROR resource_spec.py:199 -- Could not parse gpu information.
Traceback (most recent call last):
  File "/home/sc2427/jupyter-env/lib/python3.9/site-packages/ray/_private/resource_spec.py", line 193, in resolve
    gpu_types = _get_gpu_types_gputil()
  File "/home/sc2427/jupyter-env/lib/python3.9/site-packages/ray/_private/resource_spec.py", line 304, in _get_gpu_types_gputil
    gpu_list = GPUtil.getGPUs()
  File "/home/sc2427/jupyter-env/lib/python3.9/site-packages/GPUtil/GPUtil.py", line 102, in getGPUs
    deviceIds = int(vals[i])
ValueError: invalid literal for int() with base 10: "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running."
2023-09-27 15:41:32,910	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-09-27 15:41:34,344 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 3.0, 'memory': 91250

159/159 [==============================] - 0s 798us/step


INFO flwr 2023-09-27 15:41:38,906 | server.py:91 | initial parameters (loss, other metrics): 10.926995277404785, {'accuracy': 3.305600643157959}
INFO flwr 2023-09-27 15:41:38,907 | server.py:101 | FL starting
DEBUG flwr 2023-09-27 15:41:38,908 | server.py:218 | fit_round 1: strategy sampled 2 clients (out of 2)
(pid=135389) 2023-09-27 15:41:39.569893: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=135389) 2023-09-27 15:41:39.571873: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
(pid=135389) 2023-09-27 15:41:39.627818: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
(pid=135389) 2023-09-27 15:41:39.628411: I tensorflow/core/platform/cpu_feature_gu

...

36/36 [==============================] - 0s 1ms/step - loss: 0.2343 - root_mean_squared_error: 0.4841
(launch_and_fit pid=117789) Epoch 7/10
36/36 [==============================] - 0s 1ms/step - loss: 0.2335 - root_mean_squared_error: 0.4832
(launch_and_fit pid=117789) Epoch 8/10
36/36 [==============================] - 0s 2ms/step - loss: 0.2261 - root_mean_squared_error: 0.4755
(launch_and_fit pid=117789) Epoch 9/10
36/36 [==============================] - 0s 2ms/step - loss: 0.2321 - root_mean_squared_error: 0.4817
(launch_and_fit pid=117789) Epoch 10/10
159/159 [==============================] - 0s 831us/step


INFO flwr 2023-09-27 16:05:36,265 | server.py:119 | fit progress: (5, 0.20230086147785187, {'accuracy': 0.44977867603302}, 122.9811820560135)
DEBUG flwr 2023-09-27 16:05:36,266 | server.py:168 | evaluate_round 5: strategy sampled 9 clients (out of 9)


6/6 [==============================] - 0s 2ms/step - loss: 0.1997 - root_mean_squared_error: 0.4468


DEBUG flwr 2023-09-27 16:05:39,370 | server.py:182 | evaluate_round 5 received 9 results and 0 failures
DEBUG flwr 2023-09-27 16:05:39,372 | server.py:218 | fit_round 6: strategy sampled 9 clients (out of 9)


6/6 [==============================] - 0s 2ms/step - loss: 0.2100 - root_mean_squared_error: 0.4583
(launch_and_fit pid=117789) Epoch 1/10
36/36 [==============================] - 1s 3ms/step - loss: 0.2309 - root_mean_squared_error: 0.4806
(launch_and_fit pid=117789) Epoch 2/10
36/36 [==============================] - 0s 2ms/step - loss: 0.2163 - root_mean_squared_error: 0.4651
(launch_and_fit pid=117789) Epoch 3/10
36/36 [==============================] - 0s 2ms/step - loss: 0.2271 - root_mean_squared_error: 0.4765
(launch_and_fit pid=117789) Epoch 4/10
36/36 [==============================] - 0s 1ms/step - loss: 0.2279 - root_mean_squared_error: 0.4774
(launch_and_fit pid=117789) Epoch 5/10
36/36 [==============================] - 0s 1ms/step - loss: 0.2195 - root_mean_squared_error: 0.4685
(launch_and_fit pid=117789) Epoch 6/10
36/36 [==============================] - 0s 2ms/step - loss: 0.2251 - root_mean_squared_error: 0.4744
(launch_and_fit pid=117789) Epoch 7/10
36/36 [========

In [ ]:
#for each client, plot losses, rmse in separate graphs and pearson of each label in a single graph with all labels marked in the legend
import matplotlib.pyplot as plt
for i in range(len(results)):
    plt.figure(figsize=(20,10))
    plt.subplot(1,3,1)
    plt.plot(results[i]['Losses'], marker='_', color='red')
    plt.title("Losses for "+str(results[i]['clients'])+" clients")
    # plt.figure(figsize=(10,5))
    plt.plot(results_train[i]['Losses'],marker='*', color='green')
    plt.subplot(1,3,2)
    plt.plot(results[i]['rmse'])
    plt.plot(results_train[i]['rmse'], marker='*')

    plt.title("rmse for "+str(results[i]['clients'])+" clients")
    plt.subplot(1,3,3)
    for j in range(len(y_labels)):
        plt.plot(results[i]['pearson'][y_labels[j]])
    for j in range(len(y_labels)):
        plt.plot(results_train[i]['pearson'][y_labels[j]],marker='*')
    plt.title("pearson for "+str(results[i]['clients'])+" clients")
    plt.legend(y_labels)
    plt.savefig(f"{path}/{i+2}/{strat}.png")


In [ ]:
import json
for i in range(len(results)):
    with open(f"{path}/{i+2}/{strat}/"+str(results[i]['clients'])+"clients.json", "w") as outfile:
        json.dump(results[i], outfile)


In [ ]:
#load the results from the path "/results/fedAvgN" and create a dataframe with losses, rmse and average pearson of every label for each client
import json
import os
results=[]
for i in range(2,11):
    with open(f"{path}/{i}/{strat}/"+str(i)+"clients.json") as json_file:
        results.append(json.load(json_file))
import pandas as pd
df=pd.DataFrame(results)
df.head()

In [ ]:
df=pd.DataFrame(results)


In [ ]:
#add a new column named pearson_avg
df['pearson_avg']=[[] for i in range(len(df))]


In [ ]:
df.head()

In [ ]:
y_labels=list(df['pearson'][0].keys())

for i in range(df.shape[0]):
    li=[]
    for k in range(10):
        sum=0
        for j in range(len(y_labels)):
            sum=sum+df['pearson'][i][y_labels[j]][k]
        sum=sum/10
        li.append(sum)
    df['pearson_avg'][i]=li



In [ ]:
for i in range(df.shape[0]):
  # mini=min(df['Losses'][i])
  # idx=df['Losses'][i].index(mini)
  # print(idx)
  idx=-1
  df['Losses'][i]=df['Losses'][i][idx]
  df['rmse'][i]=df['rmse'][i][idx]
  df['pearson_avg'][i]=df['pearson_avg'][i][idx]


In [ ]:
df

In [ ]:
df.drop(['pearson'], axis=1, inplace=True)

In [ ]:
for i in range(df.shape[0]):
  try:
    data=pd.read_csv(f"{path}/clients{i+2}.csv")
    data.drop(["Unnamed: 0"], axis=1, inplace=True)
  except:
    data=pd.DataFrame(columns=["methods", "Loss", "RMSE", "PCC"])
  # data=data.append(pd.Series(["FedAvg_NoAug",df['Losses'][i],df['rmse'][i],df['pearson_avg'][i]]), ignore_index=True)
  data=pd.concat([data,pd.Series([f"{strat}",df['Losses'][i],df['rmse'][i],df['pearson_avg'][i]] , index=data.columns ).to_frame().T])
  data.to_csv(f"{path}/clients{i+2}.csv")

